In [1]:
import pandas as pd
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_rows', None)

In [4]:
train_df = pd.read_json('/content/drive/MyDrive/Hackaton/json/train.json', orient='index')

In [5]:
 site_gendercategory = pd.read_csv('/content/drive/MyDrive/Hackaton/site_gendercategory.csv', sep=',')

In [6]:
df = train_df

In [7]:
# male   - 1
# female - 0
df.loc[df['target'] == 'female', 'target'] = 0
df.loc[df['target'] == 'male', 'target'] = 1


In [8]:
df_total = df

In [9]:
df_total = df_total.drop(['features'], axis=1)

Проработаем exchange_sessions

In [10]:
exchange_sessions_df = df

In [11]:
exchange_sessions_df.loc[:,'exchange-sessions'] = exchange_sessions_df.loc[:,'features'].apply(lambda x: x['exchange-sessions'] if 'exchange-sessions' in x else 0)
exchange_sessions_df = exchange_sessions_df.drop('features', axis=1)

In [12]:
def get_exchange_session(data_df):

    def add_record():
        exchange_sessions_data['user-id'].append(user_id)
        exchange_sessions_data['landed-at'].append(landed_at)
        exchange_sessions_data['sites'].append(sites)
        exchange_sessions_data['accepted-site-id'].append(accepted_site_id)
        exchange_sessions_data['accepted-at'].append(accepted_at)
        exchange_sessions_data['clicks'].append(clicks)
        exchange_sessions_data['target'].append(target)

    exchange_sessions_data = {
        'user-id': [],
        'landed-at': [],
        'sites': [],
        'accepted-site-id': [],
        'accepted-at': [],
        'clicks': [],
        'target': []
    }

    users_list = data_df.index
    for i in range(data_df.shape[0]):
        if data_df['exchange-sessions'][i] == 0: continue
        user_id = users_list[i]
        # male   - 1
        # female - 0
        if data_df['target'][i] == 'male':
            target = 1
        else:
            target = 0
        for session in data_df['exchange-sessions'][i]:
            landed_at = session['landed-at']
            sites = session['sites']
            accepted_site_id = session.get('accepted-site-id', None)
            accepted_at = session.get('accepted-at', None)
            clicks = session.get('clicks', None)
            add_record()

    return(exchange_sessions_data)

exchange_sessions_df = pd.DataFrame(get_exchange_session(exchange_sessions_df))
exchange_sessions_df = exchange_sessions_df[['user-id', 'clicks']]
exchange_sessions_df['site_click'] = 0

In [13]:
exchange_sessions_df = exchange_sessions_df.dropna().reset_index(drop=True)


In [14]:
for i in range(len(exchange_sessions_df)):
  exchange_sessions_df['site_click'][i] = exchange_sessions_df['clicks'][i][0]['site-id']

<ipython-input-14-84cbc775dc51>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_sessions_df['site_click'][i] = exchange_sessions_df['clicks'][i][0]['site-id']


In [15]:
exchange_sessions_df = exchange_sessions_df.drop(['clicks'], axis=1)

In [16]:
exchange_sessions_df['gender_category'] = 2

In [17]:
site_gendercategory = site_gendercategory.set_index('site-id')

In [18]:
for i in range(len(exchange_sessions_df)):
  try:
    exchange_sessions_df['gender_category'][i] = site_gendercategory.loc[exchange_sessions_df['site_click'][i], 'gender_category']
  except:
    pass

<ipython-input-18-af3c281c3a21>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange_sessions_df['gender_category'][i] = site_gendercategory.loc[exchange_sessions_df['site_click'][i], 'gender_category']


In [19]:
# количество посещенных женских сайтов
df_total['es_female'] = 0

# количество посещенных мужских сайтов
df_total['es_male'] = 0

# количество посещенных нейтральных сайтов
df_total['es_neutrale'] = 0

In [20]:
for i in range(len(exchange_sessions_df)):
  index=exchange_sessions_df['user-id'][i]

  if exchange_sessions_df['gender_category'][i] == 0:
    df_total['es_female'][index] += 1
  elif exchange_sessions_df['gender_category'][i] == 1:
    df_total['es_male'][index] += 1
  elif exchange_sessions_df['gender_category'][i] == 2:
    df_total['es_neutrale'][index] += 1

<ipython-input-20-47645046d1b6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['es_neutrale'][index] += 1
<ipython-input-20-47645046d1b6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['es_male'][index] += 1
<ipython-input-20-47645046d1b6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['es_female'][index] += 1


Проработаем last-visits

In [21]:
last_visits_df = df

In [22]:
last_visits_df.loc[:,'last-visits'] = last_visits_df.loc[:,'features'].apply(lambda x: x['last-visits-in-categories'] if 'last-visits-in-categories' in x else 0)
last_visits_df = last_visits_df.drop('features', axis=1)


In [23]:
def get_last_visits(data_df):

    def add_record():
        last_visits_data['user-id'].append(user_id)
        last_visits_data['category'].append(category)
        last_visits_data['last-visit-at'].append(last_visit_at)
        last_visits_data['target'].append(target)

    last_visits_data = {
        'user-id': [],
        'category': [],
        'last-visit-at': [],
        'target': []
    }

    users_list = data_df.index
    for i in range(data_df.shape[0]):
        if data_df['last-visits'][i] == 0: continue
        user_id = users_list[i]
        # male   - 1
        # female - 0
        if data_df['target'][i] == 'male':
            target = 1
        else:
            target = 0
        for last_visit in data_df['last-visits'][i]:
            category = last_visit['category']
            last_visit_at = last_visit['last-visit-at']

            add_record()

    return(last_visits_data)

last_visits_df = pd.DataFrame(get_last_visits(last_visits_df))

In [24]:
last_visits_df.head()

,user-id,category,last-visit-at,target
0,user_2,other,1696278323,0
1,user_3,electronics,1686982782,0
2,user_3,other,1696320204,0
3,user_4,hypermarket,1693902771,0
4,user_4,other,1698567835,0


In [25]:
visit_category_name = ['other', 'electronics', 'hypermarket', 'bank', 'entertainment_services','fashion', 'media', 'cosmetics', 'food', 'education', 'furniture', 'health', 'travel', 'kids', 'household_appliances','luxury', 'pets', 'shoes', 'insurance', 'gifts', 'sport', 'dealoftheday', 'software']

In [26]:
for i in visit_category_name:
  df_total[f'last_visit_{i}'] = 0

In [27]:
for i in range(len(last_visits_df)):
  index = last_visits_df['user-id'][i]
  for cvn in visit_category_name:
    if last_visits_df['category'][i] == cvn:
        df_total[f'last_visit_{cvn}'][index] += 1

<ipython-input-27-a5676ec2ab5b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total[f'last_visit_{cvn}'][index] += 1


Проработаем visits

In [28]:
visits_df = df

In [29]:
visits_df.loc[:,'visits'] = visits_df.loc[:,'features'].apply(lambda x: x['visits'] if 'visits' in x else 0)
visits_df = visits_df.drop('features', axis=1)

In [30]:
def get_visits_df(data_df):
    def add_record(user_id, site_id, first_seen, last_seen, session_duration,
                   visited_at, pages_count, visited_items, visited_general_categories,
                   visited_universal_brands, target):
        visits_data['user-id'].append(user_id)
        visits_data['site-id'].append(site_id)
        visits_data['first-seen'].append(first_seen)
        visits_data['last-seen'].append(last_seen)
        visits_data['visited-at'].append(visited_at)
        visits_data['session-duration'].append(session_duration)
        visits_data['pages-count'].append(pages_count)
        visits_data['visited-items'].append(visited_items)
        visits_data['visited-general-categories'].append(visited_general_categories)
        visits_data['visited-universal_brands'].append(visited_universal_brands)
        visits_data['target'].append(target)

    visits_data = {'user-id': [], 'site-id': [], 'first-seen': [], 'last-seen': [], 'visited-at': [],
                   'session-duration': [], 'session-duration': [], 'pages-count': [], 'visited-items': [],
                   'visited-general-categories': [], 'visited-universal_brands': [], 'target': []}
    users_list = data_df.index
    for i in range(data_df.shape[0]):
        if data_df['visits'][i] == 0: continue
        user_id = users_list[i]
        # male   - 1
        # female - 0
        if data_df['target'][i] == 'male':
            target = 1
        else:
            target = 0
        for site_visits in data_df['visits'][i]:
            site_id = site_visits['site-id']
            first_seen = site_visits['first-seen']
            last_seen = site_visits['last-seen']
            for visit in site_visits['visits']:
                visited_at = visit.get('visited-at', None)
                session_duration = visit.get('session-duration', None)
                pages_count = visit.get('pages-count', None)
                visited_items = visit.get('visited-items', None)
                visited_general_categories = visit.get('visited-general-categories', None)
                visited_universal_brands = visit.get('visited-universal-brands', None)

                add_record(user_id, site_id, first_seen, last_seen, session_duration,
                   visited_at, pages_count, visited_items, visited_general_categories,
                   visited_universal_brands, target)

    return(visits_data)


visits_df = pd.DataFrame(get_visits_df(visits_df))

In [31]:
visits_df['gender_category'] = 2

In [32]:
for i in range(len(visits_df)):
  try:
    visits_df['gender_category'][i] = site_gendercategory.loc[visits_df['site-id'][i], 'gender_category']
  except:
    pass

<ipython-input-32-bf68a5be942a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visits_df['gender_category'][i] = site_gendercategory.loc[visits_df['site-id'][i], 'gender_category']


In [33]:
visits_df.head(10)


,user-id,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal_brands,target,gender_category
0,user_2,3,1696277805,1696278323,1696277805,0,1,None,None,None,0,2
1,user_2,3,1696277805,1696278323,1696278323,112,2,None,None,None,0,2
2,user_3,3,1696320204,1696320204,1696320204,0,1,None,None,None,0,2
3,user_3,23,1686372887,1686899354,1686372887,296,13,[item_2],None,None,0,1
4,user_3,23,1686372887,1686899354,1686805600,37,5,[item_2],None,None,0,1
5,user_3,23,1686372887,1686899354,1686899354,99,5,None,None,None,0,1
6,user_3,16,1680410306,1686982782,1681145345,558,6,"[item_3, item_4, item_5]","[91461, 198119, 91491]","[1, 921]",0,2
7,user_3,16,1680410306,1686982782,1686373121,93,3,None,None,None,0,2
8,user_3,16,1680410306,1686982782,1686711421,1189,10,"[item_6, item_7]","[91461, 198119, 91491]",[1],0,2
9,user_3,16,1680410306,1686982782,1686805443,105,2,None,None,None,0,2


In [34]:
df_total['visit_female_sites'] = 0
df_total['visit_male_sites'] = 0
df_total['visit_neutrale_sites'] = 0
df_total['total_pages_count'] = 0

In [35]:
for i in range(len(visits_df)):
  index = visits_df['user-id'][i]
  if visits_df['gender_category'][i] == 0:
    df_total['visit_female_sites'][index] += 1
  elif visits_df['gender_category'][i] == 1:
    df_total['visit_male_sites'][index] += 1
  elif visits_df['gender_category'][i] == 2:
    df_total['visit_neutrale_sites'][index] += 1



<ipython-input-35-b74c7d13011e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['visit_neutrale_sites'][index] += 1
<ipython-input-35-b74c7d13011e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['visit_male_sites'][index] += 1
<ipython-input-35-b74c7d13011e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['visit_female_sites'][index] += 1


In [36]:
for i in range(len(visits_df)):
  index = visits_df['user-id'][i]
  df_total['total_pages_count'][index] += visits_df['pages-count'][i]

<ipython-input-36-41fd76449df8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['total_pages_count'][index] += visits_df['pages-count'][i]


In [37]:
df_total['mean_pages_count'] =  df_total['total_pages_count']/(df_total['visit_female_sites']+df_total['visit_male_sites']+df_total['visit_neutrale_sites'])

In [38]:
df_total = df_total.drop('total_pages_count', axis=1)

Проработаем orders

In [39]:
orders_df = df

In [40]:
orders_df.head(10)

,target,features,exchange-sessions,last-visits,visits
user_1,0,"{'orders': [{'site-id': 1, 'orders': [{'create...",0,0,0
user_2,0,"{'visits': [{'site-id': 3, 'first-seen': 16962...",0,"[{'category': 'other', 'last-visit-at': 169627...","[{'site-id': 3, 'first-seen': 1696277805, 'las..."
user_3,1,"{'orders': [{'site-id': 21, 'orders': [{'creat...",0,"[{'category': 'electronics', 'last-visit-at': ...","[{'site-id': 3, 'first-seen': 1696320204, 'las..."
user_4,1,"{'orders': [{'site-id': 2, 'orders': [{'create...",0,"[{'category': 'hypermarket', 'last-visit-at': ...","[{'site-id': 3, 'first-seen': 1693468067, 'las..."
user_5,0,"{'orders': [{'site-id': 39, 'orders': [{'creat...",0,"[{'category': 'insurance', 'last-visit-at': 16...","[{'site-id': 3, 'first-seen': 1693796766, 'las..."
user_6,0,"{'orders': [{'site-id': 123, 'orders': [{'crea...","[{'landed-at': 1698263812, 'sites': [169, 214,...","[{'category': 'furniture', 'last-visit-at': 16...","[{'site-id': 21, 'first-seen': 1698263297, 'la..."
user_7,0,"{'visits': [{'site-id': 225, 'first-seen': 169...",0,"[{'category': 'hypermarket', 'last-visit-at': ...","[{'site-id': 225, 'first-seen': 1690061422, 'l..."
user_8,0,"{'orders': [{'site-id': 196, 'orders': [{'crea...",0,"[{'category': 'other', 'last-visit-at': 169757...","[{'site-id': 3, 'first-seen': 1688302122, 'las..."
user_9,0,"{'orders': [{'site-id': 307, 'orders': [{'crea...",0,"[{'category': 'fashion', 'last-visit-at': 1696...","[{'site-id': 236, 'first-seen': 1689873835, 'l..."
user_10,1,"{'orders': [{'site-id': 49, 'orders': [{'creat...",0,"[{'category': 'bank', 'last-visit-at': 1688631...","[{'site-id': 3, 'first-seen': 1698348639, 'las..."


In [41]:
orders_df.loc[:,'orders'] = orders_df.loc[:,'features'].apply(lambda x: x['orders'] if 'orders' in x else 0)
orders_df = orders_df.drop('features', axis=1)

In [42]:
def get_orders_df(data_df):
    def add_record(user_id, site_id, created_at, item_id, count,
                   general_category_path, brand_id, target):
        orders_data['user-id'].append(user_id)
        orders_data['site-id'].append(site_id)
        orders_data['created-at'].append(created_at)
        orders_data['item-id'].append(item_id)
        orders_data['count'].append(count)
        orders_data['general-category-path'].append(general_category_path)
        orders_data['brand-id'].append(brand_id)
        orders_data['target'].append(target)

    orders_data = {'user-id': [], 'site-id': [], 'created-at': [], 'item-id': [], 'count': [],
                   'general-category-path': [], 'brand-id': [], 'target': []}
    users_list = data_df.index
    for i in range(data_df.shape[0]):
        if data_df['orders'][i] == 0: continue
        user_id = users_list[i]
        # male   - 1
        # female - 0
        if data_df['target'][i] == 'male':
            target = 1
        else:
            target = 0
        for site_orders in data_df['orders'][i]:
            site_id = site_orders['site-id']
            for order in site_orders['orders']:
                created_at = order.get('created-at', None)
                if order['items']:
                    for item in order['items']:
                        item_id = item.get('id', None)
                        count = item.get('count', None)
                        general_category_path = item.get('general-category-path', None)
                        brand_id = item.get('brand-id', None)
                        add_record(user_id, site_id, created_at, item_id, count,
                                    general_category_path, brand_id, target)
                else:
                    item_id = None
                    count = None
                    general_category_path = None
                    brand_id = None
                    add_record(user_id, site_id, created_at, item_id, count,
                                general_category_path, brand_id, target)
    return(orders_data)


orders_df = pd.DataFrame(get_orders_df(orders_df))

In [43]:
orders_df['1_level_category'] = 0
orders_df['2_level_category'] = 0
orders_df['3_level_category'] = 0
orders_df['4_level_category'] = 0
orders_df['5_level_category'] = 0

In [44]:
for i in range(len(orders_df)):
  try:
    orders_df['1_level_category'][i] = orders_df['general-category-path'][i][-1]
  except:
    pass

<ipython-input-44-fb047e2e34a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['1_level_category'][i] = orders_df['general-category-path'][i][-1]


In [45]:
for i in range(len(orders_df)):
  try:
    orders_df['2_level_category'][i] = orders_df['general-category-path'][i][-2]
  except:
    pass

<ipython-input-45-c39bfddcb4de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['2_level_category'][i] = orders_df['general-category-path'][i][-2]


In [46]:
for i in range(len(orders_df)):
  try:
    orders_df['3_level_category'][i] = orders_df['general-category-path'][i][-3]
  except:
    pass

<ipython-input-46-4c7f39bec61c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['3_level_category'][i] = orders_df['general-category-path'][i][-3]


In [47]:
for i in range(len(orders_df)):
  try:
    orders_df['4_level_category'][i] = orders_df['general-category-path'][i][-4]
  except:
    pass

<ipython-input-47-0b40f609fc0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['4_level_category'][i] = orders_df['general-category-path'][i][-4]


In [48]:
for i in range(len(orders_df)):
  try:
    orders_df['5_level_category'][i] = orders_df['general-category-path'][i][-5]
  except:
    pass

<ipython-input-48-e1c9b611dc66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['5_level_category'][i] = orders_df['general-category-path'][i][-5]


In [49]:
level_1_categories = pd.read_csv('/content/drive/MyDrive/Hackaton/level_1_categories.csv', sep=',')
level_2_categories = pd.read_csv('/content/drive/MyDrive/Hackaton/level_2_categories.csv', sep=',')
level_3_categories = pd.read_csv('/content/drive/MyDrive/Hackaton/level_3_categories.csv', sep=',')
level_4_categories = pd.read_csv('/content/drive/MyDrive/Hackaton/level_4_categories.csv', sep=',')
level_5_categories = pd.read_csv('/content/drive/MyDrive/Hackaton/level_5_categories.csv', sep=',')

In [50]:
orders_df.head(10)

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target,1_level_category,2_level_category,3_level_category,4_level_category,5_level_category
0,user_1,1,1634292444,None,NaN,None,NaN,0,0,0,0,0,0
1,user_1,2,1639827297,None,NaN,None,NaN,0,0,0,0,0,0
2,user_1,2,1639827681,None,NaN,None,NaN,0,0,0,0,0,0
3,user_3,21,1684248081,None,NaN,None,NaN,0,0,0,0,0,0
4,user_3,22,1684892551,item_1,1.0,None,NaN,0,0,0,0,0,0
5,user_4,2,1600677045,item_8,1.0,None,NaN,0,0,0,0,0,0
6,user_4,29,1611987662,item_9,1.0,"[90404, 90403, 90402]",3243.0,0,90402,90403,90404,0,0
7,user_4,29,1655426430,item_10,1.0,"[90698, 7330336, 90692, 90666]",1210537.0,0,90666,90692,7330336,90698,0
8,user_4,29,1655426430,item_11,1.0,"[8353924, 91497, 91461, 198119]",1773.0,0,198119,91461,91497,8353924,0
9,user_4,30,1681785620,None,NaN,None,NaN,0,0,0,0,0,0


In [51]:
level_1_categories = level_1_categories.set_index('category_id')
level_2_categories = level_2_categories.set_index('category_id')
level_3_categories = level_3_categories.set_index('category_id')
level_4_categories = level_4_categories.set_index('category_id')
level_5_categories = level_5_categories.set_index('category_id')

In [52]:
orders_df['gender_category'] = 2


In [53]:
for i in range(len(orders_df)):
  if orders_df['5_level_category'][i] != 0:
    orders_df['gender_category'][i] = level_5_categories.loc[orders_df['5_level_category'][i], 'gender_category']
  elif orders_df['4_level_category'][i] != 0:
   orders_df['gender_category'][i] = level_4_categories.loc[orders_df['4_level_category'][i], 'gender_category']
  elif orders_df['3_level_category'][i] != 0:
   orders_df['gender_category'][i] = level_3_categories.loc[orders_df['3_level_category'][i], 'gender_category']
  elif orders_df['2_level_category'][i] != 0:
    orders_df['gender_category'][i] = level_2_categories.loc[orders_df['2_level_category'][i], 'gender_category']
  elif orders_df['1_level_category'][i] != 0:
   orders_df['gender_category'][i] = level_1_categories.loc[orders_df['1_level_category'][i], 'gender_category']
  else:
    orders_df['gender_category'][i] = 3


<ipython-input-53-e998d6a93986>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['gender_category'][i] = 3
<ipython-input-53-e998d6a93986>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['gender_category'][i] = level_3_categories.loc[orders_df['3_level_category'][i], 'gender_category']
<ipython-input-53-e998d6a93986>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['gender_category'][i] = level_4_categor

In [54]:
orders_df['gender_category'].value_counts()

3    1832527
2     853829
0     755469
1      16573
Name: gender_category, dtype: int64

In [55]:
df_total['order_female'] = 0
df_total['order_male'] = 0
df_total['order_neutrale'] = 0

In [56]:
for i in range(len(orders_df)):
  index = orders_df['user-id'][i]
  if orders_df['gender_category'][i] == 0:
    df_total['order_female'][index] += 1
  elif orders_df['gender_category'][i] == 1:
    df_total['order_male'][index] += 1
  elif orders_df['gender_category'][i] == 2:
    df_total['order_neutrale'][index] += 1
  else:
    pass

<ipython-input-56-166982d5a815>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['order_neutrale'][index] += 1
<ipython-input-56-166982d5a815>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['order_female'][index] += 1
<ipython-input-56-166982d5a815>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['order_male'][index] += 1


In [57]:
df_total['target'] = df_total['target'].astype('int32')

In [58]:
df_total.to_csv('/content/drive/MyDrive/Hackaton/df_total.csv')

Обучение модели

In [59]:
# Итоговый датасет для обучения
df_total.head()

,target,es_female,es_male,es_neutrale,last_visit_other,last_visit_electronics,last_visit_hypermarket,last_visit_bank,last_visit_entertainment_services,last_visit_fashion,...,last_visit_sport,last_visit_dealoftheday,last_visit_software,visit_female_sites,visit_male_sites,visit_neutrale_sites,mean_pages_count,order_female,order_male,order_neutrale
user_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,0
user_2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,2,1.500000,0,0,0
user_3,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,3,11,4.571429,0,0,0
user_4,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,6,11,3.176471,0,0,3
user_5,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,1,22,5.391304,0,0,3


In [60]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127755 entries, user_1 to user_127755
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   target                             127755 non-null  int32  
 1   es_female                          127755 non-null  int64  
 2   es_male                            127755 non-null  int64  
 3   es_neutrale                        127755 non-null  int64  
 4   last_visit_other                   127755 non-null  int64  
 5   last_visit_electronics             127755 non-null  int64  
 6   last_visit_hypermarket             127755 non-null  int64  
 7   last_visit_bank                    127755 non-null  int64  
 8   last_visit_entertainment_services  127755 non-null  int64  
 9   last_visit_fashion                 127755 non-null  int64  
 10  last_visit_media                   127755 non-null  int64  
 11  last_visit_cosmetics              

In [61]:
df_total = df_total.dropna()

In [62]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [63]:
X = df_total.drop('target', axis=1)
y = df_total.target

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [65]:
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic'
    , n_estimators=250
    , learning_rate=0.10
    , colsample_bytree=0.70
    , max_depth=3
    , n_jobs=4
)

In [66]:
xgb_model.fit(
    X_train, y_train
    , early_stopping_rounds=5
    , eval_set=[(X_test, y_test)]
    , verbose=False
)

print('Training Done!')

predictions = xgb_model.predict(X_test)

print('Predictions Done!')

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Training Done!
Predictions Done!


In [67]:
accuracy = accuracy_score(y_test, predictions)
train_score = round(xgb_model.score(X_train, y_train) * 100, 2)
valid_score = round(xgb_model.score(X_test, y_test) * 100, 2)

print('Accuracy: ', round(accuracy * 100, 2), '%', sep='')

Accuracy: 73.86%
